# Estudo sobre Doação de sangue

## 1. Sobre o Projeto

Nesse projeto, buscamos pesquisar e entender a similaridade entre doadores de sangue e a construção de um algoritmo capaz de predizer a probabilidade de uma pessoa ser doadora de sangue.
Para isso, usamos a base de dados Predicting Blood Donor Availability *(disponíel em : https://www.kaggle.com/datasets/sumedh1507/blood-donor-dataset?resource=download)* para elaboração desse projeto.

Essa base de dados possui 10 mil entradas, com os seguintes atributos:
- Id de doador
- Nome
- Email
- Senha
- Telefone
- Cidade
- Tipo sanguíneo
- Disponibilidade
- Meses desde a primeira doação
- Número de doações
- Pints (500ml) doados
- Data de criação

## 1.1 Metodologia

Como nosso objetivo principal é chegarmos a uma predição de uma nova entrada, irems estabeler algumas regras para execução desse estudo.

Para nos auxiliar com o estudo, iremos focar apenas nas propriedades: Tipo sanguíneo, Disponibilidade, Meses desde a primeira doação, número de doações e data de criação.

## 1.2 Agrupamento

Para auxiliar na predição do nosso modelo, agruparemos nossos dados em dois grupos:
- Doadores ativos
- Doadores inativos

Para fazermos essa classificação, iremos nos basear na condição biológica do corpo humano de esperar entre 60-90 dias para uma nova doação. Isso significa que em 1 ano, o uma pessoa pode doar 6 vezes no máximo.

*Referência: [Dia Mundial do Doador de Sangue: 8 coisas que você precisa saber antes de doar](https://portal.pucrs.br/noticias/saude/como-doar-sangue/)*

Diante disso, classificaremos os doadores ativos como aqueles que fizeram mais de 3 doações por ano e todos os outros como inativos.

Isso será importante para classificação dos doadores na nossa base de dados

# 2. Algoritmo do projeto

### Variáveis globais

In [ ]:
DATASET_PATH = "./dataset/blood_donor_dataset.csv"

## 2.1 Importação de bibliotecas

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

## 2.2 Atribuição da base de dados

In [ ]:
df = pd.read_csv(DATASET_PATH)
df

# 3. Pré processamento de dados

Como dito anteriormente, iremos remover alguns dados irrelevantes para a parametrização do nosso (nome, telefone, cidade, etc...). Acreditamos que essas propriedades não impactam/ não fazem sentido para o problema proposto.

In [ ]:
df = df.drop("donor_id", axis="columns")
df = df.drop("name", axis="columns")
df = df.drop("email", axis="columns")
df = df.drop("password", axis="columns")
df = df.drop("contact_number", axis="columns")
df = df.drop("city", axis="columns")
df = df.drop("pints_donated", axis="columns")
df = df.drop("created_at", axis="columns")
df

Além disso, iremos reestruturar a proridade disponibilidade para valores numéricos

In [ ]:
df['availability'] = df['availability'].replace({'Yes': True, 'No': False})
df

Agora iremos aplicar o One Hot Enconding para a propriedade de Tipos sanguíneos

In [ ]:
df = pd.get_dummies(df, columns=['blood_group'])
df

Criando a coluna de doador ativo

In [ ]:
df["years_since_first_donation"] = df["months_since_first_donation"] / 12
df["donations_per_year"] = df["number_of_donation"] / df["years_since_first_donation"]
df["active_donor"] = (df["donations_per_year"] > 3).astype(int)
df

Removendo outliers (Pessoas que doaram mais que 6x num ano)

In [ ]:
df = df[
    (df["number_of_donation"] / (df["months_since_first_donation"] / 12)) <= 6
].copy()
df

# 4. Criando algoritomos de classificação

Preparando a propriedade alvo

In [ ]:
X = df.drop(columns=["active_donor"])
y = df["active_donor"]
X = X.astype(int)

## 4.1 Árvore Binária

In [ ]:
clf = DecisionTreeClassifier(max_depth=3, random_state=42)
clf.fit(X, y)

plt.figure(figsize=(20, 10))
plot_tree(clf, feature_names=X.columns, class_names=["Inativo", "Ativo"], filled=True, fontsize=12)
plt.show()

### Teste da arvore binária

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42
)

clf = DecisionTreeClassifier(max_depth=3, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy:.2%}")

## 4.2 KNN

In [ ]:
# Definindo dataset de teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42
)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print(f"Acurácia: {accuracy_score(y_test, y_pred):.2%}")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))
print("\nMatriz de Confusão:\n", pd.DataFrame(confusion_matrix(y_test, y_pred),
      index=["Real Inativo", "Real Ativo"],
      columns=["Previsto Inativo", "Previsto Ativo"]))

## 4.3 Regressão logística

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42
)

In [ ]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_proba = log_reg.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

auc_score = roc_auc_score(y_test, y_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC - Classificação de Doadores Ativos')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()
print(f"AUC: {auc_score:.2f}")